In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install torchtext==0.9.0

In [8]:
help(torchtext.legacy.datasets.LanguageModelingDataset.splits)

Help on method splits in module torchtext.legacy.data.dataset:

splits(path=None, root='.data', train=None, validation=None, test=None, **kwargs) method of builtins.type instance
    Create Dataset objects for multiple splits of a dataset.
    
    Arguments:
        path (str): Common prefix of the splits' file paths, or None to use
            the result of cls.download(root).
        root (str): Root dataset storage directory. Default is '.data'.
        train (str): Suffix to add to path for the train set, or None for no
            train set. Default is None.
        validation (str): Suffix to add to path for the validation set, or None
            for no validation set. Default is None.
        test (str): Suffix to add to path for the test set, or None for no test
            set. Default is None.
        Remaining keyword arguments: Passed to the constructor of the
            Dataset (sub)class being used.
    
    Returns:
        Tuple[Dataset]: Datasets for train, validati

In [10]:
sen1

['新华社雅典月日电记者刘咏秋希腊政府日宣布希腊总理齐普拉斯兼任外交部长接替日辞去外长职务的科恰斯',
 '博尔顿上月在接受哥伦比亚广播公司采访谈到同朝鲜建立互信以及对去核进行核实的问题他提到了利比亚模式',
 '塑料也有好处不仅仅是经济上的也是环境上的',
 '据称椰子油包治百病从口臭到消化不良',
 '其实她不至于一句日语也不会',
 '法医陈庆用解剖刀还原真相中新网',
 '法国总统府同日说法国总统马克龙日与特朗普进行电话会谈重申中导条约对于欧洲安全以及战略稳定的重要性',
 '本版块活动为公益活动不向参加者推荐医疗服务或者其他商品相关信息仅供参考',
 '随后记者致电上海小额贷款公司协会咨询对方表示协会里没有乔融金融服务有限公司也从未听说过该公司',
 '这是克里姆林宫办公厅副主任的看法',
 '当杰克慢慢地陷入疯狂的时候他结交了几个凶残的鬼魂最终说服他去杀死自己的妻子和儿子至死方休',
 '进一步创新贸易监管制度',
 '他察觉到对方的驾驶方式很危险但当时也没多想',
 '但也有人不太高兴',
 '突发脑梗后小时最关键',
 '这与企业的产品宣传不无相关',
 '原南京铁路公安处长江大桥派出所民警钱士成就亲眼见证了这段历史',
 '同济大学建筑与城市规划学院党委书记彭震伟主持纪念大会建筑与城市规划学院院长李振宇介绍陈从周先生的生平事迹',
 '同时根据中华人民共和国道路交通安全法有关规定王某被处以记分罚款的处罚',
 '女子组冠军则被美国选手杰林考夫摘得',
 '莫怕他人笑掉牙但愿儿时笑常在',
 '新华社记者安蓓姜潇许晟',
 '王兴质押在摩拜科技全部股权摩拜称股权质押是架构的标准流程之一此前在美团上市招股书中已披露',
 '即便被几十家媒体团团围住但殡仪馆外的整个氛围依旧肃穆庄严',
 '何念台',
 '陈少波表示扶贫干部不仅要做到扶贫知识熟记于心更要做到因户施策运用自如充分运用帮扶政策攻坚克难消除贫困',
 '崔永元本人也在微博上附上该网民连日发布的威胁内容有的甚至写道崔永元你墓地都看好啦遗照都选好了最好别出门内部消息有人出万要你手脚',
 '当时的香港唱片工业虽然蓬勃但市场多由偶像歌星的翻唱歌曲占领原创摇滚音乐的空间非常有限',
 '他告诫乘客不管因何种私事都应遵守交通规则',
 '只要其中任何一名法官因为回避或其他原因不能参加审判上诉机构必定无法

In [14]:
import torchtext
from torchtext.vocab import Vectors
import torch
import torch.nn as nn
import numpy as np
import random
import re
from torchtext.legacy.data import Field
def preprocessing(sent):
    pattern = re.compile(r'([\u4e00-\u9fa5])')
    chars = "".join(pattern.findall(sent))
    #chars =[w for w in chars if len(w) > 0]
    return chars

with open('/content/drive/MyDrive/英文文档NLP/data.txt','r',encoding='utf-8') as file:
    content=file.readlines()
sen1=[]
for i in range(len(content)):
  sen=preprocessing(content[i])
  if len(sen)>0:
    sen1.append(sen)
with open('train.txt','w',encoding='utf-8') as file:
  for s in sen1[0:1000]:
    file.writelines(s+'\n')
  file.close

with open('dev.txt','w',encoding='utf-8') as file:
  for s in sen1[1000:1500]:
    file.write(s+'\n')
    file.close
  
with open('test.txt','w',encoding='utf-8') as file:
  for s in sen1[1500:2000]:
    file.write(s+'\n')
    file.close
#np.savetxt('dev.txt',str(sen1[1000:1500]))
#np.savetxt('test.txt',sen1[1500:2000])

batch_size = 32
embedding_size = 50
max_vocab_size = 50000
hidden_size = 100
learn_rate = 0.001

TEXT = Field(lower=True)
# 专门用来处理语言模型数据集
train,val,test=torchtext.legacy.datasets.LanguageModelingDataset.splits(path=".",  #当前文件夹
                    train="train.txt", 
                    validation="dev.txt", 
                    test="test.txt",text_field=TEXT)
TEXT.build_vocab(train,max_size=max_vocab_size)          # 从train数据集中建立词表，按词频建立
# print(TEXT.vocab.itos[:100])
# print(TEXT.vocab.stoi['<unk>'])
 
train_iter,val_iter,test_iter = torchtext.legacy.data.BPTTIterator.splits((train,val,test),
                                batch_size=batch_size,bptt_len=50,repeat=False,shuffle=True)
# it=iter(trainiter)
# batch = next(it)
 


In [15]:
class RNNmodel(nn.Module):
    def __init__(self,vocab_size,embed_size,hidden_size):
        super(RNNmodel,self).__init__()
        self.embed = nn.Embedding(vocab_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size)     # 通过设置batch_first = True ，输入可以是 batch_size*seq_length其他不用变
        self.decoder = nn.Linear(hidden_size,vocab_size)
        self.hidden_size = hidden_size
 
    def forward(self,text,hidden):
        emb = self.embed(text)       # text: seq_length*batch_size
        output,hidden = self.lstm(emb,hidden) # output: seq_length*batch_size*hidden_size   hidden: (1*batch_size*hidden_size,1*batch_size*hidden_size  )
        decoded = self.decoder(output.view(-1,output.size(-1)))
        decoded = decoded.view(output.size(0),output.size(1),decoded.size(1))  # decoded: seq_length * batch_size * vocab_size
        return decoded, hidden
 
 
    def init_hidden(self,bsize,requires_grad=True):
        weight = next(self.parameters())
        return (weight.new_zeros((1,bsize,self.hidden_size),requires_grad=requires_grad),
                weight.new_zeros((1,bsize,self.hidden_size),requires_grad=requires_grad))
 
 

In [16]:
model = RNNmodel(len(TEXT.vocab),embedding_size,hidden_size)
optimizer=torch.optim.Adam(model.parameters(),lr=learn_rate)
loss_fn = nn.CrossEntropyLoss()
 
 
def repackage_hidden(h):
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
 
 
def evaluate(model,data):
    model.eval()
    total_loss = 0.
    total_count = 0.
    it = iter(data)
    with torch.no_grad():
        hidden = model.init_hidden(batch_size,requires_grad=False)
        for i, batch in enumerate(it):
            data, target = batch.text, batch.target
            hidden = repackage_hidden(hidden)
 
            output, hidden = model(data, hidden)
            loss = loss_fn(output.view(-1, len(TEXT.vocab)), target.view(-1))
            total_loss = loss.item()*np.multiply(*data.size())
            total_count = np.multiply(*data.size())
    loss = total_loss/total_count
    model.train()
    return loss
 
val_losses = []
GRAD_CLIP =5.0
for epoch in range(2):
    model.train()
    it = iter(train_iter)
    hidden = model.init_hidden(batch_size)
    for i, batch in enumerate(it):
        data,target = batch.text,batch.target
        hidden = repackage_hidden(hidden)
        output,hidden = model(data,hidden)
        loss = loss_fn(output.view(-1, len(TEXT.vocab)),target.view(-1))  # batch_size * target_class_dim    batch_size
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(),GRAD_CLIP)        # 优化方法
        loss.backward()
        optimizer.step()
        schedule = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)         # 调整学习率来优化，0.5即降一半
        if i % 100 == 0:
            print('loss',loss.item())
        if i % 1000 == 0:
            val_loss = evaluate(model,val_iter)
            if len(val_losses)==0 or val_loss < min(val_losses):
                torch.save(model.state_dict(),'lm.pth')
                print('best model saved to lm.pth')
            else:
                schedule.step()
                optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
            val_losses.append(val_loss)
 
 
 

loss 6.938003063201904
best model saved to lm.pth
loss 6.870462417602539
best model saved to lm.pth


In [19]:
print("perplexity: ", np.exp(val_loss))

perplexity:  930.8939872232444


In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/英文文档NLP/RNNLM.pkl' )